In [1]:
import pandas as pd
from datasets import Dataset
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
import tensorflow as tf

c:\Users\dweep\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [50]:
# Load your dataset
df = pd.read_csv('intents.csv')

In [51]:
df.shape

(97, 2)

In [52]:

df.tail()

,text,intent
92,Find the items in the store.,Object detection
93,Can you identify the objects on the floor?,Object detection
94,What are the objects in this picture?,Object detection
95,Detect any items in the photo.,Object detection
96,Identify the objects in this snapshot.,Object detection


In [53]:

# Encode the labels
df['label'] = df['intent'].factorize()[0]

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('google/bert_uncased_L-2_H-128_A-2')

# Tokenize the dataset
def preprocess_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Convert Datasets to TensorFlow Format
train_tf_dataset = train_dataset.to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["label"],
    shuffle=True,
    batch_size=16,
    collate_fn=lambda x: {key: tf.convert_to_tensor([elem[key] for elem in x]) for key in x[0]}
)

val_tf_dataset = val_dataset.to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["label"],
    shuffle=False,
    batch_size=16,
    collate_fn=lambda x: {key: tf.convert_to_tensor([elem[key] for elem in x]) for key in x[0]}
)


Map: 100%|██████████| 20/20 [00:00<00:00, 809.71 examples/s]
c:\Users\dweep\AppData\Local\Programs\Python\Python310\lib\site-packages\datasets\arrow_dataset.py:410: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [54]:
from transformers import TFBertForSequenceClassification

# Load the TinyBERT model
model = TFBertForSequenceClassification.from_pretrained('google/bert_uncased_L-2_H-128_A-2', num_labels=2)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
# Train the model
model.fit(train_tf_dataset, validation_data=val_tf_dataset, epochs=20)


Epoch 1/20
5/5 [==============================] - 16s 511ms/step - loss: 0.6993 - sparse_categorical_accuracy: 0.4416 - val_loss: 0.6744 - val_sparse_categorical_accuracy: 0.8000
Epoch 2/20
5/5 [==============================] - 1s 177ms/step - loss: 0.6739 - sparse_categorical_accuracy: 0.5714 - val_loss: 0.6506 - val_sparse_categorical_accuracy: 0.7500
Epoch 3/20
5/5 [==============================] - 1s 161ms/step - loss: 0.6589 - sparse_categorical_accuracy: 0.6364 - val_loss: 0.6259 - val_sparse_categorical_accuracy: 0.7500
Epoch 4/20
5/5 [==============================] - 1s 164ms/step - loss: 0.6342 - sparse_categorical_accuracy: 0.6234 - val_loss: 0.5980 - val_sparse_categorical_accuracy: 0.8000
Epoch 5/20
5/5 [==============================] - 1s 154ms/step - loss: 0.6111 - sparse_categorical_accuracy: 0.7143 - val_loss: 0.5695 - val_sparse_categorical_accuracy: 0.8000
Epoch 6/20
5/5 [==============================] - 1s 165ms/step - loss: 0.5920 - sparse_categorical_accuracy:

In [56]:
# Save the fine-tuned model
model.save_pretrained('path_to_save_model')
tokenizer.save_pretrained('path_to_save_model')


('path_to_save_model\\tokenizer_config.json',
 'path_to_save_model\\special_tokens_map.json',
 'path_to_save_model\\vocab.txt',
 'path_to_save_model\\added_tokens.json')

In [57]:
model_path = 'path_to_save_model'  # Replace with your actual path
tokenizer = BertTokenizer.from_pretrained(model_path)
model = TFBertForSequenceClassification.from_pretrained(model_path)

Some layers from the model checkpoint at path_to_save_model were not used when initializing TFBertForSequenceClassification: ['dropout_55']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at path_to_save_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [58]:
def predict_intent(text):
    # Preprocess the input text
    inputs = tokenizer(text, return_tensors='tf', padding=True, truncation=True, max_length=128)
    
    # Make predictions
    outputs = model(inputs)
    predictions = tf.nn.softmax(outputs.logits, axis=-1)
    
    # Get the predicted label
    predicted_label = tf.argmax(predictions, axis=1).numpy()[0]
    
    # Map label indices to intent names
    label_map = {0: 'Read command', 1: 'Object detection'}
    
    return label_map[predicted_label], predictions.numpy()[0]



In [64]:

# Example usage
new_text = "see the name of that sign"
predicted_intent, prediction_scores = predict_intent(new_text)

print(f"Predicted intent: {predicted_intent}")
print(f"Prediction scores: {prediction_scores}")

Predicted intent: Read command
Prediction scores: [0.58779645 0.41220352]
